In [11]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle

In [12]:
# train = pd.read_pickle(FILE.train_base_line.value)
train = pd.read_pickle(FILE.train_agg_v3_select.value)
print('train shape is: {}'.format(train.shape))
train_append = pd.read_pickle(FILE.train_agg_v3_mean.value)
print('train_append shape is: {}'.format(train_append.shape))
# test = pd.read_pickle(FILE.test_base_line.value)
test = pd.read_pickle(FILE.test_agg_v3_select.value)
print('test shape is: {}'.format(test.shape))
test_append = pd.read_pickle(FILE.test_agg_v3_mean.value)
print('test_append shape is: {}'.format(test_append.shape))


train shape is: (1001650, 35)
train_append shape is: (1001650, 160)
test shape is: (40024, 34)
test_append shape is: (40024, 159)


# LGBM

In [16]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, target='click'):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df[target].mean(), holdout[target].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df[target].notnull()]
        test_df = df[df[target].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in [target,'time','instance_id','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df[target])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df[target].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df[target].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=18,
            n_estimators=30000,
            learning_rate=lr,
            device='cpu',
#             max_bin =10000,
#             num_leaves=num_leaves,
#             colsample_bytree=colsample, # 0.67
#             subsample=subsample,
#             subsample_freq=0, ## disable subsampling
#             max_depth=max_depth,
#             reg_alpha=0.65,
#             reg_lambda=reg_lambda,
#             min_split_gain=0.0222415,
#             min_child_weight=39.3259775,
#             min_child_samples=min_child_samples,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 200)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout[target], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = log_loss(holdout[target], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        print('saving...')
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))
    
    

def select_holdout(train,test,shift=1,seed=6):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [17]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [20]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.05]
    nfolds = 5
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=None, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=True)                                        

In [18]:
def add_loop_each(train,test,train_append,test_append,file_name='add_each_col_score_0915.csv',ignore_list = ['click','instance_id','time'],order_col='instance_id'):
    assert np.sum(train[order_col].values != train_append[order_col].values) == 0
    assert np.sum(test[order_col].values != test_append[order_col].values) == 0
    try:
        df_add_result = pd.read_csv(file_name)
        processed_list = list(df_add_result['colName'].values)
        best = df_add_result.sort_values(['val_mean','holdout_overall']).iloc[0]
        best_ho_full = best['holdout_overall']
        best_ho_mean = best['holdout_mean']
        best_cv_mean = best['val_mean']
        
    except:
        df_add_result = pd.DataFrame()
        processed_list = []
        best_ho_full = 0
        best_ho_mean = 0 
        best_cv_mean = 0
    need_process_list = list(set(list(train_append.columns))-set(train.columns)-set(processed_list)-set(ignore_list))
    need_process_list = ['noadd'] + need_process_list
    counter = 1
    for col in need_process_list:
        print('processing {}, {}/{}'.format(col,counter,len(need_process_list)))
        if col in processed_list:
            continue
            counter += 1
        if col not in ignore_list:
            train_add = train.copy()
            test_add = test.copy()
            if col != 'noadd':
                train_add[col] = train_append[col].copy()
                test_add[col] = test_append[col].copy()
 
            train_df,holdout = select_holdout(train_add,test_add,shift=1,seed=41)
            print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
            print('train mean: {}, holdout mean: {}'.format(train_df.click.mean(),holdout.click.mean()))
            feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test_add, holdout)
            score_new_df = pd.DataFrame({'colName':[col],
                                         'holdout_overall':[holdout_roc],
                                         'holdout_mean':[holdout_mean],
                                         'val_mean':[full_te_mean],
                                         'train_mean':[full_tr_mean]})
            df_add_result = pd.concat([df_add_result,score_new_df],ignore_index=True,sort=False)
            df_add_result.to_csv(file_name,index=False)
            processed_list.append(col)
            counter+=1
            
            best = df_add_result.sort_values(['val_mean','holdout_overall']).iloc[0]
            best_ho_full = best['holdout_overall']
            best_ho_mean = best['holdout_mean']
            best_cv_mean = best['val_mean']
            print('best holdout overall score is: {}'.format(best_ho_full))
            print('best holdout mean score is: {}'.format(best_ho_mean))
            print('best cv mean score is: {}'.format(best_cv_mean))
    return df_add_result
            
        

In [26]:
file_name_formater = 'add_each_col_score_v3_mean_round{}.csv'
dump_name = 'add_cols_0915_v3_mean.pkl'
order_col = 'instance_id'

assert np.sum(train[order_col].values != train_append[order_col].values) == 0
assert np.sum(test[order_col].values != test_append[order_col].values) == 0 
try:
    add_list = pickle.load(dump_name,'rb')
except:
    add_list = []
r = len(add_list)+1
for col in add_list:
    train[col] = train_append[col].copy()
    test[col] = test_append[col].copy()

while True:
    print('round {} ...'.format(r))
    df_add_result = add_loop_each(train,test,train_append,test_append,file_name_formater.format(r))
    df_add_result_base = df_add_result[df_add_result['colName']=='noadd'].iloc[0]
    base_ho = df_add_result_base['holdout_overall']
    base_cv = df_add_result_base['val_mean']
    df_add_result_better = df_add_result[(df_add_result['holdout_overall']<base_ho) &(df_add_result['val_mean']<base_cv)].copy()
    if len(df_add_result_better) == 0:
        print('finish')
        break
    else:
        df_add_result_better = df_add_result_better.sort_values(['val_mean','holdout_overall'])
        col = df_add_result_better.iloc[0]['colName']
        train[col] = train_append[col].copy()
        test[col] = test_append[col].copy()
        add_list.append(col)
        pickle.dump(add_list,open(dump_name,'wb'))
        r+=1
        print('current add list is:{}'.format(add_list))

round 1 ...
processing noadd, 1/1
current add list is:['model+creative_type+app_id_target_mean']
round 2 ...
processing noadd, 1/1
current add list is:['model+creative_type+app_id_target_mean', 'osv+model+creative_type+app_id+advert_name_target_mean']
round 3 ...
processing noadd, 1/1
current add list is:['model+creative_type+app_id_target_mean', 'osv+model+creative_type+app_id+advert_name_target_mean', 'osv+model+creative_type_target_mean']
round 4 ...
processing noadd, 1/1
current add list is:['model+creative_type+app_id_target_mean', 'osv+model+creative_type+app_id+advert_name_target_mean', 'osv+model+creative_type_target_mean', 'user_tags+creative_type+app_id+city_target_mean']
round 5 ...
processing noadd, 1/1
current add list is:['model+creative_type+app_id_target_mean', 'osv+model+creative_type+app_id+advert_name_target_mean', 'osv+model+creative_type_target_mean', 'user_tags+creative_type+app_id+city_target_mean', 'model+advert_name_target_mean']
round 6 ...
processing noadd, 1

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [ 1100] AUC : ho: 0.426675 / te: 0.415478 / tr: 0.398045 (diff: 0.011197)
Fold  2 [  613] AUC : ho: 0.425988 / te: 0.414525 / tr: 0.405381 (diff: 0.011463)
Fold  3 [  747] AUC : ho: 0.425760 / te: 0.415111 / tr: 0.403053 (diff: 0.010649)
Fold  4 [  721] AUC : ho: 0.425673 / te: 0.416189 / tr: 0.403368 (diff: 0.009485)
Fold  5 [  800] AUC : ho: 0.426145 / te: 0.415101 / tr: 0.402442 (diff: 0.011043)
Full HO score 0.425364
FULL HO mean 0.426048, std 0.000355
FULL TE mean 0.415281, std 0.000547
FULL TR mean 0.402458, std 0.002417
FULL DIFF mean 0.010768, std 0.000694
Run LightGBM with kfold - done in 105s
best holdout ov

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [ 1126] AUC : ho: 0.426742 / te: 0.415367 / tr: 0.397566 (diff: 0.011375)
Fold  2 [  707] AUC : ho: 0.426072 / te: 0.414467 / tr: 0.403918 (diff: 0.011605)
Fold  3 [  911] AUC : ho: 0.426061 / te: 0.415021 / tr: 0.400622 (diff: 0.011039)
Fold  4 [  758] AUC : ho: 0.425773 / te: 0.416178 / tr: 0.402879 (diff: 0.009595)
Fold  5 [  800] AUC : ho: 0.426168 / te: 0.415131 / tr: 0.402423 (diff: 0.011038)
Full HO score 0.425437
FULL HO mean 0.426163, std 0.000318
FULL TE mean 0.415233, std 0.000557
FULL TR mean 0.401482, std 0.002229
FULL DIFF mean 0.010930, std 0.000701
Run LightGBM with kfold - done in 111s
best holdout ov

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [ 1127] AUC : ho: 0.426913 / te: 0.415422 / tr: 0.397556 (diff: 0.011491)
Fold  2 [  899] AUC : ho: 0.426265 / te: 0.414586 / tr: 0.401232 (diff: 0.011679)
Fold  3 [  990] AUC : ho: 0.425865 / te: 0.415026 / tr: 0.399619 (diff: 0.010838)
Fold  4 [  785] AUC : ho: 0.426275 / te: 0.416275 / tr: 0.402448 (diff: 0.010001)
Fold  5 [  826] AUC : ho: 0.426235 / te: 0.415128 / tr: 0.402103 (diff: 0.011107)
Full HO score 0.425532
FULL HO mean 0.426311, std 0.000338
FULL TE mean 0.415287, std 0.000562
FULL TR mean 0.400592, std 0.001805
FULL DIFF mean 0.011023, std 0.000589
Run LightGBM with kfold - done in 115s
best holdout ov

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  942] AUC : ho: 0.426936 / te: 0.415467 / tr: 0.400281 (diff: 0.011469)
Fold  2 [ 1268] AUC : ho: 0.426405 / te: 0.414460 / tr: 0.395889 (diff: 0.011945)
Fold  3 [ 1144] AUC : ho: 0.426425 / te: 0.414956 / tr: 0.397373 (diff: 0.011469)
Fold  4 [  748] AUC : ho: 0.425931 / te: 0.416208 / tr: 0.402961 (diff: 0.009723)
Fold  5 [  628] AUC : ho: 0.426193 / te: 0.415042 / tr: 0.404964 (diff: 0.011151)
Full HO score 0.425564
FULL HO mean 0.426378, std 0.000331
FULL TE mean 0.415227, std 0.000586
FULL TR mean 0.400293, std 0.003372
FULL DIFF mean 0.011151, std 0.000758
Run LightGBM with kfold - done in 118s
best holdout ov

...

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  825] AUC : ho: 0.426442 / te: 0.415488 / tr: 0.401785 (diff: 0.010954)
Fold  2 [  718] AUC : ho: 0.426063 / te: 0.414639 / tr: 0.403596 (diff: 0.011424)
Fold  3 [  894] AUC : ho: 0.425884 / te: 0.415112 / tr: 0.400720 (diff: 0.010772)
Fold  4 [  483] AUC : ho: 0.425819 / te: 0.416364 / tr: 0.406961 (diff: 0.009454)
Fold  5 [  909] AUC : ho: 0.426348 / te: 0.415048 / tr: 0.400610 (diff: 0.011300)
Full HO score 0.425443
FULL HO mean 0.426111, std 0.000247
FULL TE mean 0.415330, std 0.000583
FULL TR mean 0.402734, std 0.002370
FULL DIFF mean 0.010781, std 0.000703
Run LightGBM with kfold - done in 101s
best holdout ov

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  931] AUC : ho: 0.426662 / te: 0.415515 / tr: 0.400364 (diff: 0.011147)
Fold  2 [  600] AUC : ho: 0.426467 / te: 0.414438 / tr: 0.405519 (diff: 0.012029)
Fold  3 [ 1031] AUC : ho: 0.425847 / te: 0.415007 / tr: 0.398882 (diff: 0.010840)
Fold  4 [  736] AUC : ho: 0.425685 / te: 0.416219 / tr: 0.403163 (diff: 0.009466)
Fold  5 [  766] AUC : ho: 0.426225 / te: 0.415073 / tr: 0.402951 (diff: 0.011152)
Full HO score 0.425473
FULL HO mean 0.426177, std 0.000367
FULL TE mean 0.415250, std 0.000593
FULL TR mean 0.402176, std 0.002318
FULL DIFF mean 0.010927, std 0.000831
Run LightGBM with kfold - done in 106s
best holdout ov

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  992] AUC : ho: 0.426459 / te: 0.415356 / tr: 0.399240 (diff: 0.011103)
Fold  2 [  792] AUC : ho: 0.426462 / te: 0.414471 / tr: 0.402560 (diff: 0.011991)
Fold  3 [ 1139] AUC : ho: 0.425916 / te: 0.414955 / tr: 0.397324 (diff: 0.010962)
Fold  4 [ 1184] AUC : ho: 0.425957 / te: 0.416092 / tr: 0.396688 (diff: 0.009865)
Fold  5 [  514] AUC : ho: 0.425896 / te: 0.415079 / tr: 0.406777 (diff: 0.010817)
Full HO score 0.425342
FULL HO mean 0.426138, std 0.000264
FULL TE mean 0.415191, std 0.000534
FULL TR mean 0.400518, std 0.003738
FULL DIFF mean 0.010947, std 0.000679
Run LightGBM with kfold - done in 116s
best holdout ov

train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 41) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  974] AUC : ho: 0.426692 / te: 0.415558 / tr: 0.399712 (diff: 0.011134)
Fold  2 [  694] AUC : ho: 0.426098 / te: 0.414493 / tr: 0.404133 (diff: 0.011605)
Fold  3 [  659] AUC : ho: 0.425714 / te: 0.414998 / tr: 0.404464 (diff: 0.010716)
Fold  4 [  787] AUC : ho: 0.425908 / te: 0.416188 / tr: 0.402427 (diff: 0.009721)
Fold  5 [  503] AUC : ho: 0.425990 / te: 0.415220 / tr: 0.407044 (diff: 0.010770)
Full HO score 0.425439
FULL HO mean 0.426080, std 0.000331
FULL TE mean 0.415292, std 0.000566
FULL TR mean 0.403556, std 0.002424
FULL DIFF mean 0.010789, std 0.000621
Run LightGBM with kfold - done in 93s
best holdout ove

KeyboardInterrupt: 

In [9]:
# processing noadd, 1/124
# train shape: (961626, 40), holdout shape: (40024, 40)
# train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
# #############################################
# 0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
# #############################################
# (961626, 40) (40024, 39) (40024, 40)
# MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
# Fold  1 [ 1016] AUC : ho: 0.426748 / te: 0.415512 / tr: 0.399426 (diff: 0.011236)
# Fold  2 [  935] AUC : ho: 0.426371 / te: 0.414450 / tr: 0.400809 (diff: 0.011920)
# Fold  3 [ 1017] AUC : ho: 0.425713 / te: 0.414939 / tr: 0.399368 (diff: 0.010773)
# Fold  4 [  909] AUC : ho: 0.426202 / te: 0.416160 / tr: 0.400900 (diff: 0.010043)
# Fold  5 [  535] AUC : ho: 0.426194 / te: 0.415111 / tr: 0.406628 (diff: 0.011083)
# Full HO score 0.425466
# FULL HO mean 0.426245, std 0.000334
# FULL TE mean 0.415235, std 0.000575
# FULL TR mean 0.401426, std 0.002681
# FULL DIFF mean 0.011011, std 0.000613
# Run LightGBM with kfold - done in 84s
# best holdout overall score is: 0.42546649165002576
# best holdout mean score is: 0.4262454816155887
# best cv mean score is: 0.41523454521160297

In [24]:
add_list

['model+creative_type+app_id_target_mean',
 'osv+model+creative_type+app_id+advert_name_target_mean',
 'osv+model+creative_type_target_mean',
 'user_tags+creative_type+app_id+city_target_mean',
 'model+advert_name_target_mean',
 'osv+user_tags+creative_type+app_id+advert_name_target_mean']

In [25]:
train.to_pickle(FILE.train_agg_v3_select.value)
test.to_pickle(FILE.test_agg_v3_select.value)

In [10]:
df_add_result.sort_values(['holdout_overall'])

,colName,holdout_overall,holdout_mean,val_mean,train_mean
134,model+app_id+osv_cumulative_count,0.426177,0.426995,0.416452,0.406367
107,model+app_id+advert_name_cumulative_count,0.426253,0.427043,0.416455,0.406482
0,noadd,0.426321,0.426978,0.416367,0.407271
48,app_id+city+advert_name_cumulative_count,0.426330,0.426986,0.416499,0.407990
3,app_id+advert_name+creative_type_time_to_n_next,0.426350,0.426989,0.416515,0.407722
101,user_tags+creative_type+osv_count,0.426351,0.426987,0.416474,0.407889
77,advert_name+creative_type+osv_time_to_n_next,0.426365,0.427075,0.416561,0.406969
43,model+city+osv_count,0.426385,0.427042,0.416472,0.407477
161,advert_name+user_tags+creative_type_reverse_cu...,0.426396,0.427005,0.416457,0.407751
133,model+app_id+advert_name_count,0.426399,0.426965,0.416501,0.408748
